In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import json
from matplotlib import pyplot as plt
import glob
import os
import paddleocr
from paddleocr import PaddleOCR
from paddleocr import draw_ocr
from PIL import Image
import numpy as np
import math
import sys
import cv2
import gdown
import tarfile

sys.path.append('../../')
from ocr_ensemble.label_studio_utils import convert_labelstudio2json

In [3]:
with open('../../data/label_studio/done.json', 'r') as f:
    labelled = json.load(f)

In [4]:
labelled[0]

{'id': 1761,
 'annotations': [{'id': 389,
   'completed_by': 1,
   'result': [{'original_width': 1024,
     'original_height': 1024,
     'image_rotation': 0,
     'value': {'x': 15.686274509803921,
      'y': 70.83333333333334,
      'width': 68.38235294117648,
      'height': 7.352941176470589,
      'rotation': 0},
     'id': 'dJ8fOIEo2B',
     'from_name': 'bbox',
     'to_name': 'image',
     'type': 'rectangle',
     'origin': 'manual'},
    {'original_width': 1024,
     'original_height': 1024,
     'image_rotation': 0,
     'value': {'x': 15.686274509803921,
      'y': 70.83333333333334,
      'width': 68.38235294117648,
      'height': 7.352941176470589,
      'rotation': 0,
      'text': ['GUDNEX ENTERPRISES']},
     'id': 'dJ8fOIEo2B',
     'from_name': 'transcription',
     'to_name': 'image',
     'type': 'textarea',
     'origin': 'manual'},
    {'original_width': 1024,
     'original_height': 1024,
     'image_rotation': 0,
     'value': {'x': 35.618937944897034,
      '

In [5]:
json_dataset = convert_labelstudio2json(labelled)

In [6]:
len(json_dataset)

443

In [45]:
json_dataset[0]

{'ocr_annotation': {'bounding_boxes': [[[160.62745098039215,
     725.3333333333335],
    [860.8627450980393, 725.3333333333335],
    [860.8627450980393, 800.6274509803923],
    [160.62745098039215, 800.6274509803923]],
   [[364.7379245557456, 802.6046279679395],
    [655.9024843694806, 802.6046279679395],
    [655.9024843694806, 835.0806750240869],
    [364.7379245557456, 835.0806750240869]],
   [[170.93044822256567, 728.0370942812983],
    [851.4868624420402, 728.0370942812983],
    [851.4868624420402, 794.5100463678516],
    [170.93044822256567, 794.5100463678516]],
   [[370.34930448222565, 804.0061823802164],
    [661.564142194745, 804.0061823802164],
    [661.564142194745, 838.8253477588871],
    [370.34930448222565, 838.8253477588871]]],
  'text': ['GUDNEX ENTERPRISES',
   'POSTED ON JIJI',
   'GUDNEX ENTERPRISES',
   'POSTED ON JIJI'],
  'rotation': [0, 0, 0, 0]},
 'width': 1024,
 'height': 1024,
 'label_studio_fname': '/data/upload/1/60569e35-013370001.jpg',
 'label_studio_id':

# check whether we can render the bounding boxes

In [61]:
ls_folder = '../../data/label_studio/'
image_folder = ls_folder + '01337/'

In [62]:
file_id = '1jdbv0-6msFbl5AoKdNviQAgaNQLETr10'
url = f'https://drive.google.com/uc?id={file_id}'
output = ls_folder+'01337.tar'
tar_file_path = gdown.download(url, output, quiet=False)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1jdbv0-6msFbl5AoKdNviQAgaNQLETr10
From (redirected): https://drive.google.com/uc?id=1jdbv0-6msFbl5AoKdNviQAgaNQLETr10&confirm=t&uuid=9d6b8123-73d2-4d7c-8ae6-cbeeb037867c
To: C:\Users\Chris Wendler\Documents\git\2023\OCR-ensemble\data\label_studio\01337.tar
100%|█████████████████████████████████████████████████████████████████████████████| 1.16G/1.16G [00:11<00:00, 99.9MB/s]


In [63]:
with tarfile.open(tar_file_path, 'r') as tar_ref:
    tar_ref.extractall(image_folder)

In [64]:
def my_draw_ocr(imname, annotation,font_path='../../fonts/simfang.ttf'):
    bboxes = annotation['bounding_boxes']
    txts = annotation['text']
    scores = annotation['confidences'] if 'confidences' in annotation else [1]*len(txts)
    im_show = draw_ocr(Image.open(imname), bboxes, txts, scores, font_path=font_path)
    im_show = Image.fromarray(im_show)
    im_show.show()

In [65]:
n_max = 10
i = 0
for d in json_dataset:
    imname = image_folder + d['label_studio_fname'].split('-')[1]
    if sum(d['ocr_annotation']['rotation']) != 0:
        my_draw_ocr(imname, d['ocr_annotation'])
        i += 1
        if i >= n_max:
            break

In [66]:
import cv2

def convert_box_to_rectangle(box):
    top_left, top_right, bottom_right, bottom_left = box
    # Compute the center as the average of all points
    center = np.mean(box, axis=0)
    # Compute the width and height as the distance between points
    width = np.linalg.norm(np.array(top_right) - np.array(top_left))
    height = np.linalg.norm(np.array(top_right) - np.array(bottom_right))
    # Compute the angle as the arctan of the ratio of the height and width
    angle = np.arctan(height / width)
    return (center, (width, height), angle)


def create_mask(bounding_boxes, mask_size):
    mask = np.zeros(mask_size)
    for box in bounding_boxes:
        # Convert the box points to integer type
        box_int = np.array(box, dtype=np.int32)
        # Reshape to match the shape expected by cv2.fillPoly (must be 3-dimensional)
        reshaped = box_int.reshape((-1, 1, 2))
        # Draw the rotated rectangle on the mask
        cv2.fillPoly(mask, [reshaped], color=255)
    return mask


def intersection_mask(mask1, mask2):
    return np.logical_and(mask1, mask2)

def union_mask(mask1, mask2):
    return np.logical_or(mask1, mask2)

def iou_score(d1, d2, w, h, plot=False):
    bb1 = d1['bounding_boxes']
    bb2 = d2['bounding_boxes']
    bb1_mask = create_mask(bb1, (h, w))
    bb2_mask = create_mask(bb2, (h, w))
    if plot:
        plt.imshow(bb1_mask)
        plt.show()
        plt.imshow(bb2_mask)
        plt.show()
    intersection = intersection_mask(bb1_mask, bb2_mask)
    union = union_mask(bb1_mask, bb2_mask)
    return np.sum(intersection) / np.sum(union)

In [67]:
ious = []
n_ocr = 0
n_correct_text_detection = 0
for d in json_dataset:
    imname = image_folder + d['label_studio_fname'].split('-')[1]
    json_name = image_folder + d['label_studio_fname'].split('-')[1].replace('.jpg', '.json')
    gt = d['ocr_annotation']
    try:
        with open(json_name, 'r') as f:
            pred = json.load(f)
        if len(gt['text']) > 0 and len(pred['ocr_result']['text']) > 0:
            n_correct_text_detection += 1
        if len(gt['text']) == 0 and len(pred['ocr_result']['text']) == 0:
            n_correct_text_detection += 1
        if len(gt['text']) > 0:
            w, h = pred['width'], pred['height']
            iou = iou_score(gt, pred['ocr_result'], w, h)
            ious += [iou]
            #print(imname, iou)
            n_ocr += 1
    except FileNotFoundError as e:
        if len(gt['text']) == 0:
            n_correct_text_detection += 1
        else: 
            ious += [0]
        continue
print(np.mean(ious), 'over', n_ocr)
print(n_correct_text_detection / len(json_dataset), 'accuracy in text detection')

0.5568636967680889 over 223
0.8961625282167043 accuracy in text detection


In [68]:
len(json_dataset)

443

In [69]:
from ocr_ensemble.proposers import rotatedCrop

In [70]:
from p_tqdm import p_map

def process_data(d, rgb_folder=image_folder, rotatedCrop=rotatedCrop):
    from PIL import Image
    import numpy as np
    imname = rgb_folder + d['label_studio_fname'].split('-')[1]
    gt = d['ocr_annotation']
    crops = []
    labels = []
    img = Image.open(imname)
    img = np.array(img)
    for bbox, txt in zip(gt['bounding_boxes'], gt['text']):
        crop, crop_180 = rotatedCrop(img, bbox, return_180=True)
        crops += [crop, crop_180]
        labels += 2*[txt]
    return crops, labels



# Use p_map to apply the function to each item in json_dataset
results = p_map(process_data, json_dataset)# num_cpus=6)

# Now, results is a list of tuples, where each tuple contains a list of crops and a list of labels.
# We need to combine these into single lists of crops and labels.
crops, labels = zip(*results)
crops = [item for sublist in crops for item in sublist]
labels = [item for sublist in labels for item in sublist]


  0%|          | 0/443 [00:00<?, ?it/s]

crops = []
labels = []
for d in tqdm(json_dataset):
    imname = rgb_folder + d['label_studio_fname'].split('-')[1]
    gt = d['ocr_annotation']
    # make crops
    img = Image.open(imname)
    img = np.array(img)
    for bbox, txt in zip(gt['bounding_boxes'], gt['text']):
        crop, crop_180 = rotatedCrop(img, bbox, return_180=True)
        crops += [crop]
        crops += [crop_180]
        labels += 2*[txt]

In [72]:
crops_bgr = [np.array(cv2.cvtColor(np.array(crop.convert('RGB')), cv2.COLOR_RGB2BGR)) for crop in crops]

In [73]:
from tqdm import tqdm

ocr = PaddleOCR(lang='en', use_angle_cls=True)

batch_size = 100
batch = []
predictions = []
confidences = []
for crop in tqdm(crops_bgr):
    batch += [crop]
    if len(batch) % batch_size == 0:
        preds = ocr.ocr(batch, det=False, cls=False)
        for pred in preds:
            predictions += [pred[0][0]]
            confidences += [pred[0][1]]
            if len(pred) > 1:
                print('len > 1!!!')
        batch = []
if len(batch) > 0:
    preds = ocr.ocr(batch, det=False, cls=False)
    for pred in preds:
        predictions += [pred[0][0]]
        confidences += [pred[0][1]]
    batch = []

[2023/05/24 16:22:31] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Chris Wendler/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Chris Wendler/.paddleocr/whl\\rec\\en\\en_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_te

100%|██████████████████████████████████████████████████████████████████████████████| 4498/4498 [09:30<00:00,  7.89it/s]


In [81]:
len(predictions)

4498

In [88]:
better_predictions = []
better_labels = []
better_confidences = []
better_crops = []
for i, (pred, conf, crop) in enumerate(zip(predictions[::2], confidences[::2], crops_bgr[::2])):
    idx = 2*i
    if confidences[idx+1] > conf:
        better_predictions += [predictions[idx+1]]
        better_confidences += [confidences[idx+1]]
        better_labels += [labels[idx+1]]
        better_crops += [crops_bgr[idx+1]]
    else:
        better_predictions += [pred]
        better_confidences += [conf]
        better_labels += [labels[idx]]
        better_crops += [crop]

In [89]:
len(better_labels)

2249

In [90]:
len(better_predictions)

2249

In [91]:
len(better_crops)

2249

In [92]:
better_crops[0]

array([[[187, 211, 209],
        [194, 216, 214],
        [196, 217, 215],
        ...,
        [174, 203, 208],
        [149, 173, 179],
        [135, 159, 165]],

       [[193, 215, 213],
        [189, 211, 209],
        [186, 207, 205],
        ...,
        [ 96, 122, 128],
        [ 37,  59,  65],
        [ 27,  46,  53]],

       [[201, 223, 221],
        [192, 214, 212],
        [186, 207, 205],
        ...,
        [102, 128, 134],
        [ 24,  46,  52],
        [  0,  17,  24]],

       ...,

       [[191, 221, 216],
        [189, 219, 214],
        [177, 207, 202],
        ...,
        [238, 253, 255],
        [232, 245, 247],
        [239, 254, 255]],

       [[195, 225, 220],
        [194, 224, 219],
        [181, 211, 206],
        ...,
        [241, 254, 255],
        [231, 244, 246],
        [241, 254, 255]],

       [[214, 244, 239],
        [201, 231, 226],
        [180, 210, 205],
        ...,
        [236, 249, 251],
        [232, 244, 246],
        [241, 254, 255]]

In [86]:
from weighted_levenshtein import lev
levs = []
for gt, pred in zip(better_labels, better_predictions):
    print(gt.encode("ascii", "ignore").decode(), ',', pred.encode("ascii", "ignore").decode())
    levs += [lev(gt.encode("ascii", "ignore").decode(), pred.encode("ascii", "ignore").decode())]
print(np.mean(levs))

GUDNEX ENTERPRISES , GUDNEXENTERPRISES
POSTED ON JIJI , POSTEDONIJI
GUDNEX ENTERPRISES , GUDNEXENTERPRIOES
POSTED ON JIJI , POSTEDONOJI
RUMBLE , RUMBLE
RUMBLE , RUMBLE
WHOOOM , Maoon
GOOD BYE. , GOOD BYE
DANNY, AND , DANNY.AND
GOOD GODDAMN , GOOD GODDAMN
RIDDANCE. , RIDDANCE.
50% , 50%
10:28 , 1028
06 Thursday ,  06 Thursday
casall , casall
casall , casall
HARRY POTTER , Haty Potter
NOW , NOW
MUGGLES , MUGGLES
CAN BECOME , CAN BECOM
INVISIBLE , INVISIBLE
GLORY , CLORY
FINEST , FINEST
FABRIC , FABRIC
& , 8
DETAILING , DETAILING
THE , THE
CLOAK , CLOAK
INVISIBILITY , INVISIBILITY
EyeEm , EyeEm
EyeEm , EyeEm
87 , 87
OWSKI , OWSKI
87 , 87
OWSKI , OWSKI
5-MINUTE CRAFTS , 5-MINUTE CRAFTS
5-MINUTE CRAFTS , 5-MINUTE CRAFTS
f , f
THE , THE
PASTA , PASTA
Cruiser , Bruiser
FOOD TRUCK , FOODTRUCK
PLAY THE ,  PLAY THE
ROAD , ROAD
LACOSTE , LACOSTE
WHEN PAUL COMES TO, HE FINDS HIMSELF LYING , WHEN PAL COMES TO HE FINDS HIMSELF LYING
AND , AND
HE'S NOWHERE IN , HE'S NOWHERE IN
WITH HIS HEAD IN , WITH

In [87]:
# Only run this if you are patient... I also serialized it 
# ../../data/label_studio/chatgpt35_postprocessing.pkl

In [122]:
from ocr_ensemble.postprocessing import postprocess
import openai
import time
predictions_chatgpt35 = []
openai.api_key = "your api key"

In [135]:
len(predictions_chatgpt35)

168

In [136]:
print(predictions_chatgpt35)

['GUDNEX ENTERPRISES', 'POSTED ON IJI', 'GUDNEXENTERPRIOES', 'posted onoji', 'Rumble', 'Rumble.', 'Maoon', 'goodbye', 'DANNY AND', 'Good goddamn.', 'RIDDANCE.', '50%', '1028', '06 Thursday', 'casall', 'casall', 'Harry Potter', 'NOW', 'Muggles', 'CAN BECOME', 'invisible', 'CLORY', 'Finest.', 'Fabric', '8', 'Detailing', 'THE', 'CLOAK', 'Invisibility.', 'Eye Em', 'Eye Em', '87', 'OWSKI', '87', 'OWSKI', '5-MINUTE CRAFTS', '5-minute crafts', 'f', 'The', 'pasta', 'Bruiser', 'Food truck.', 'PLAY THE', 'ROAD', 'Lacoste.', 'WHEN PAL COMES TO HE FINDS HIMSELF LYING', 'AND', "HE'S NOWHERE IN", 'WITH HIS HEAD IN', "WHERE'S", "SIGHT! WE'RE LEAVING", 'HS', "WIFE'S", 'LAP.', 'WH-WHERE', 'AM', 'CHAD?', 'This place.', 'RIGHT', 'I-.?', 'I am on.', 'Pall and Diane are soon.', "We'll get to.", 'Back, Diane!', 'PACKED AND...', 'The station.', 'THERE', 'HE', 'Just in time', 'AGAIN!', 'For the train.', 'To London!', 'WHAT DO WE?', "We'll", 'HAVE', 'TO', 'Good news, darling!', 'DO NOW', 'WAIT', 'UNTIL', 'WIFE

In [138]:
while len(predictions_chatgpt35) < len(better_predictions):
    try:
        for pred in tqdm(better_predictions[len(predictions_chatgpt35):]):
            post = postprocess(pred)
            predictions_chatgpt35 += [post]
    except Exception as e:
        print(e)
        time.sleep(61)
        continue

  2%|████                                                                                                                                                                                                     | 43/2100 [02:27<1:57:33,  3.43s/it]


That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 757d4141792bb9878d7a3fd04fad4ec3 in your message.)


  0%|▏                                                                                                                                                                                                         | 2/2057 [00:33<9:31:58, 16.70s/it]


That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 147f69a88f926c276ec8e5d88d288655 in your message.)


  2%|███▍                                                                                                                                                                                                     | 35/2055 [01:11<1:08:49,  2.04s/it]


...

 18%|█████████████████████████████████████                                                                                                                                                                        | 15/83 [00:45<03:25,  3.02s/it]


That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 73fd6e56b7b8cba89fbf6f1f51ff850f in your message.)


 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                     | 61/68 [01:25<00:09,  1.40s/it]


That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 3b8dfdf9f6c85afef546f010e14b8fcd in your message.)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:07<00:00,  1.06s/it]


In [140]:
levs35 = []
for gt, pred in zip(better_labels, predictions_chatgpt35):
    print(gt.encode("ascii", "ignore").decode(), ',', pred.encode("ascii", "ignore").decode())
    levs35 += [lev(gt.encode("ascii", "ignore").decode(), pred.encode("ascii", "ignore").decode())]
print(np.mean(levs35))

GUDNEX ENTERPRISES , GUDNEX ENTERPRISES
POSTED ON JIJI , POSTED ON IJI
GUDNEX ENTERPRISES , GUDNEXENTERPRIOES
POSTED ON JIJI , posted onoji
RUMBLE , Rumble
RUMBLE , Rumble.
WHOOOM , Maoon
GOOD BYE. , goodbye
DANNY, AND , DANNY AND
GOOD GODDAMN , Good goddamn.
RIDDANCE. , RIDDANCE.
50% , 50%
10:28 , 1028
06 Thursday , 06 Thursday
casall , casall
casall , casall
HARRY POTTER , Harry Potter
NOW , NOW
MUGGLES , Muggles
CAN BECOME , CAN BECOME
INVISIBLE , invisible
GLORY , CLORY
FINEST , Finest.
FABRIC , Fabric
& , 8
DETAILING , Detailing
THE , THE
CLOAK , CLOAK
INVISIBILITY , Invisibility.
EyeEm , Eye Em
EyeEm , Eye Em
87 , 87
OWSKI , OWSKI
87 , 87
OWSKI , OWSKI
5-MINUTE CRAFTS , 5-MINUTE CRAFTS
5-MINUTE CRAFTS , 5-minute crafts
f , f
THE , The
PASTA , pasta
Cruiser , Bruiser
FOOD TRUCK , Food truck.
PLAY THE , PLAY THE
ROAD , ROAD
LACOSTE , Lacoste.
WHEN PAUL COMES TO, HE FINDS HIMSELF LYING , WHEN PAL COMES TO HE FINDS HIMSELF LYING
AND , AND
HE'S NOWHERE IN , HE'S NOWHERE IN
WITH HIS HE

In [93]:
import pickle

In [94]:
with open('../../data/label_studio/crops_bgr', 'wb') as f:
    pickle.dump(better_crops, f)

In [143]:
with open('../../data/crops_bgr.pkl', 'wb') as f:
    pickle.dump(crops_bgr, f)

In [144]:
with open('../../data/labels.pkl', 'wb') as f:
    pickle.dump(better_labels, f)

In [145]:
with open('../../data/paddle_confidences.pkl', 'wb') as f:
    pickle.dump(better_confidences, f)

In [146]:
with open('../../data/paddle_predictions.pkl', 'wb') as f:
    pickle.dump(better_predictions, f)

In [147]:
with open('../../data/chatgpt35_postprocessing.pkl', 'wb') as f:
    pickle.dump(predictions_chatgpt35, f)